#Imports

In [ ]:
import glob
import os
import shutil
import json
import csv
import networkx as nx
import matplotlib.pyplot as plt
try:
  import pyvis
  from pyvis.network import Network
except:
  !pip install pyvis
  import pyvis
  from pyvis import Network
from time import sleep
import nltk
wn = nltk.WordNetLemmatizer()
ps = nltk.PorterStemmer()
import re
import shutil
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
import string
import itertools
punctuation = string.punctuation
stopwordsset = set(stopwords.words("english"))
stopwordsset.add('rt')
stopwordsset.add("'s")
from datetime import datetime

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
WORKING_DIR = "/content/drive/MyDrive/MSDS_marketing_text_analytics/master_files/3_network_analysis"

In [ ]:
#!wget http://128.138.93.164/nikelululemonadidas_tweets.jsonl.gz -P /content/drive/MyDrive/MSDS_marketing_text_analytics/master_files/3_network_analysis

In [ ]:
#!gzip -d /content/drive/MyDrive/MSDS_marketing_text_analytics/master_files/3_network_analysis/nikelululemonadidas_tweets.jsonl.gz

#Data Prep

In [ ]:
#Removing urls
def removeURL(text):
  result = re.sub(r"http\S+", "", text)
  return result

#Extracting contextual words from a sentence
# tokenizing is taking out all the words in a sentence and turning it into tokens/words
def tokenize(text):
  #lower case
  text = text.lower()
  #split into individual words
  words = word_tokenize(text)
  return words

#stem - peaches : peach : reduce the number of repeated words
def stem(tokenizedtext):
  rootwords = []
  for aword in tokenizedtext:
    aword = ps.stem(aword)
    rootwords.append(aword)
  return rootwords

#removes useless words such as a, an, the
def stopWords(tokenizedtext):
  goodwords = []
  for aword in tokenizedtext:
    if aword not in stopwordsset:
      goodwords.append(aword)
  return goodwords

# feature reduction. taking words and getting their roots and graphing only the root words
def lemmatizer(tokenizedtext):
  lemmawords = []
  for aword in tokenizedtext:
    aword = wn.lemmatize(aword)
    lemmawords.append(aword)
  return lemmawords

#inputs a list of tokens and returns a list of unpunctuated tokens/words
def removePunctuation(tokenizedtext):
  nopunctwords = []
  for aword in tokenizedtext:
    if aword not in punctuation:
      nopunctwords.append(aword)
  cleanedwords = []
  for aword in nopunctwords:
    aword = aword.translate(str.maketrans('', '', string.punctuation))
    cleanedwords.append(aword)
    
  return cleanedwords

def removesinglewords(tokenizedtext):
  goodwords = []
  for a_feature in tokenizedtext:
    if len(a_feature) > 1:
      goodwords.append(a_feature)
  return goodwords

In [ ]:
json_file = open('%s/nikelululemonadidas_tweets.jsonl' % WORKING_DIR, 'r')

There's a lot of metadata in a tweet! If you want to take a look at all that you can get from a tweet:
https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet

In [ ]:
uniquewords = {}

for i, atweet in enumerate(json_file):
    if i % 10000 == 0:
      print(i)
    tweetjson = json.loads(atweet)
    text = tweetjson['full_text']
    #natural language pre processing : clean the tweet 
    nourlstext = removeURL(text)
    tokenizedtext = tokenize(nourlstext)
    nostopwordstext = stopWords(tokenizedtext)
    lemmatizedtext = lemmatizer(nostopwordstext)
    nopuncttext = removePunctuation(lemmatizedtext)
    
    # print(tokenizedtext)
    # print(nostopwordstext)
    # print(lemmatizedtext)
    # print(nopuncttext)
    # sleep(10)

    for aword in nopuncttext:
        if aword in uniquewords:
            uniquewords[aword] += 1
        if aword not in uniquewords:
            uniquewords[aword] = 1

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000


In [ ]:
len(uniquewords)

98470

In [ ]:
#  list of unique words: key - word; value = number of times the word appears.
wordstoinclude = set()
wordcount = 0
for aword in uniquewords:
    if uniquewords[aword] > 300: # play with this number to get around 1000 words
        wordcount += 1
        wordstoinclude.add(aword)

In [ ]:
print(wordcount)

923


In [ ]:
json_file = open('%s/nikelululemonadidas_tweets.jsonl' % WORKING_DIR, 'r')

In [ ]:
Graph = nx.Graph()

In [ ]:
uniquewords = {}

for i, atweet in enumerate(json_file):
    if i % 10000 == 0:
      print(i)
    tweetjson = json.loads(atweet)
    text = tweetjson['full_text']
    #natural language pre processing : clean the tweet 
    nourlstext = removeURL(text)
    tokenizedtext = tokenize(nourlstext)
    nostopwordstext = stopWords(tokenizedtext)
    lemmatizedtext = lemmatizer(nostopwordstext)
    nopuncttext = removePunctuation(lemmatizedtext)
    
    # print(tokenizedtext)
    # print(nostopwordstext)
    # print(lemmatizedtext)
    # print(nopuncttext)
    # sleep(10)
    
    goodwords = []
    for aword in nopuncttext:
        if aword in wordstoinclude:
            goodwords.append(aword.replace(',', ''))

    allcombos = itertools.combinations(goodwords, 2) # set this to the number of words you want in each combination
    for acombo in allcombos:
        row = []
        for anode in acombo:
            row.append(anode)
        Graph.add_edge(row[0], row[1])

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000


In [ ]:
nx.info(Graph)

'Graph with 923 nodes and 240694 edges'

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(300, 300))
#ugh these settings are such a pain, but here's the way to look them up: 
#https://networkx.org/documentation/stable/reference/generated/networkx.drawing.nx_pylab.draw_networkx.html
nx.draw_networkx(Graph, ax=ax, font_color="#FFFFFF", font_size=20, node_size=30000, width=4, arrowsize=100)
plt.savefig("%s/semantic_network.png" % WORKING_DIR, format="PNG")

/usr/local/lib/python3.7/dist-packages/matplotlib/backends/backend_agg.py:214: RuntimeWarning: Glyph 8294 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/usr/local/lib/python3.7/dist-packages/matplotlib/backends/backend_agg.py:183: RuntimeWarning: Glyph 8294 missing from current font.
  font.set_text(s, 0, flags=flags)
/usr/local/lib/python3.7/dist-packages/matplotlib/backends/backend_agg.py:214: RuntimeWarning: Glyph 8297 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/usr/local/lib/python3.7/dist-packages/matplotlib/backends/backend_agg.py:183: RuntimeWarning: Glyph 8297 missing from current font.
  font.set_text(s, 0, flags=flags)
/usr/local/lib/python3.7/dist-packages/matplotlib/backends/backend_agg.py:214: RuntimeWarning: Glyph 128293 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/usr/local/lib/python3.7/dist-packages/matplotlib/backends/backend_agg.py:183: RuntimeWarning: Glyph 128293 missing from current font.
  font.set_text(s

In [ ]:
plt.show()

In [ ]:
nt = Network('1000', '600', directed=True, notebook=True, layout='force_atlas_2based')
# populates the nodes and edges data structures
nt.from_nx(Graph)
nt.show_buttons(filter_=['physics'])
#nt.show('nx.html')
from IPython.core.display import display, HTML
display(HTML('nx.html'))